#Put your Google Colab link here:
*your link here*

### Introduction
This assignment provides hands-on experience querying different Large Language Models (LLMs) through a cloud API provider (Nebius AI). You will focus on a simplified medical Question Answering task, using sample questions from MedMCQA, a large-scale, Multiple-Choice Question Answering (MCQA) dataset designed to address real-world medical entrance exam questions.

The goal of this assignment is to develop practical skills in interacting with LLM APIs, comparing the capabilities of different open-source models (including those potentially fine-tuned for biomedical domains), exploring the impact of generation parameters, and evaluating their potential and limitations in the healthcare context.


### API platform
We will use Nebius AI Studio (https://studio.nebius.com/).

Nebius AI offers new users $1 in free credits, which should be sufficient to complete this assignment. You can monitor your usage in the Nebius AI dashboard.


## Part 1: Setup

### 1.1 Install Libraries:

In [ ]:
# Run this cell to install the OpenAI library
!pip install openai -q

### 1.2 Obtain Nebius AI API Key:

- Sign up or log in to Nebius AI Studio: https://studio.nebius.com/

- Navigate to API keys to generate an API key.
- Important: Treat your API key like a password. Do not share it publicly. Please **delete the key** before you submit the assignmnet.



### 1.3 Configure API Client
Initialize the OpenAI client to use the Nebius AI API endpoint and your key.

In [ ]:
import os
from openai import OpenAI

# TODO: Paste your Nebius AI API Key here
# Important: delete your API key before submission
NEBIUS_API_KEY = """TO DO"""

# Initialize the client to connect to Nebius AI
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/", # Nebius AI endpoint
    api_key=NEBIUS_API_KEY)


## Part 2: Load Dataset (3 points)
Install Hugging Face datasets Library: We need this library to load the MedMCQA dataset.

In [ ]:
# Run this cell to install the datasets library
# It's OK if you see some dependency conflicts
!pip install datasets

Load and Prepare [MedMCQA](https://huggingface.co/datasets/openlifescienceai/medmcqa) Dataset: We will load the validation split of the openlifescienceai/medmcqa dataset from Hugging Face, shuffle it, and select the first 100 examples for our assignment to manage API costs. We will construct the prompt based on the dataset.

In [ ]:
from datasets import load_dataset
# Define a seed for reproducibility
SEED = 42

# Load the validation split from openlifescienceai/medmcqa
print("Loading MedMCQA dataset (validation split)...")
"""TO DO"""

# Shuffle the dataset using SEED
print("Shuffling dataset...")
"""TO DO"""

# Select the first 100 examples
print("Selecting first 100 examples...")
medmcqa_sample = """TO DO"""

# Process the data into queries and true answers
queries = []
true_answers = []

for example in medmcqa_sample:
    query = f"Question:\n{"""TO DO"""}\n\nOptions:\nA. {"""TO DO"""}\nB. {"""TO DO"""}\nC. {"""TO DO"""}\nD. {"""TO DO"""}"
    answer = """TO DO""" # keep it as it is (0, 1, 2, 3)
    # append into list
    """TO DO"""

# check an example
print('query 1:')
print(queries[1])
print('answer 1:')
print(true_answers[1])

## Part 3: Query Different Models (13 points)

We will use 3 different models available on Nebius AI: Llama 3.1 8B, Llama 3.3 70B, and OpenBioLLM 70B.

Llama models are developed by Meta for general tasks, and [OpenBioLLM](https://huggingface.co/aaditya/Llama3-OpenBioLLM-70B) is finetuned from Llama for medical/biological contexts.

### 3.1 Define the function to get LLM response from API. (3 points)
You can check the example code in Nebius ai website --> Playground --> view code

In [ ]:
from openai import OpenAI

def query_llm(model_name, system_prompt, prompt, temperature=0.6, max_tokens=1000, top_p=1):
    """
    Sends a prompt to a specified OpenAI chat model and returns the generated response.

    Parameters:
        model_name (str): The name of the model.
        system_prompt (str): A message that sets the behavior or tone of the assistant.
        prompt (str): The user's input or question for the model to respond to.
        temperature (float, optional): set temperature for generation.
        max_tokens (int, optional): The maximum number of tokens in the response.
        top_p (float, optional): set top_p for generation.

    Returns:
        str: The text content of the model's response.
    """
    completion = """TO DO"""
    response_text = """TO DO"""
    return response_text


### 3.2 Query Models (3 points)
Loop through the sampled quries. For each query, use defined function to get answers from selected models.

Please use temperature 0.0.

We will prompt the model to respond in 2 different format, and evaluate the performance. Please don't change anything in the prompt.

In [ ]:
from tqdm import tqdm

models = ["meta-llama/Meta-Llama-3.1-8B-Instruct", "meta-llama/Llama-3.3-70B-Instruct-fast", "aaditya/Llama3-OpenBioLLM-70B"]

system_prompt = r"""You are a medical expert specializing in answering multiple-choice questions. \
For each question provided, carefully analyze the options (A, B, C, D) and select the most accurate answer based on your knowledge."""

# format 1: note (The correct answer is:)
format_prompt_1 = r"\nAt the end of each response, present your final answer after 'The correct answer is:'. \
For example: The correct answer is: A"

# save the responses in the following dict
responses_1 = {models[0]: [], models[1]: [], models[2]: []}

# This may take 15-20 minutes
for query in tqdm(queries, desc="runing queries..."):
    # get responses for each model
    # please use system_prompt+format_prompt_1 as system prompt, and use query+format_prompt_1 as user prompt
    """TO DO"""

import json
# Optional: save the responses
with open('responses_1.txt', 'w') as file:
    json.dump(responses_1, file)

In [ ]:
# format 2: boxed (\boxed{})
format_prompt_2 = r"\nAt the end of each response, present your final answer in \boxed{}. \
For example: \boxed{A}"

responses_2 = {models[0]: [], models[1]: [], models[2]: []}

# This may take 15-20 minutes
for query in tqdm(queries, desc="runing queries..."):
    # get responses for each model
    # please use system_prompt+format_prompt_2 as system prompt, and use query+format_prompt_2 as user prompt
    """TO DO"""

# Optional: save the responses
with open('responses_2.txt', 'w') as file:
    json.dump(responses_2, file)

### 3.3 Define Processing Functions (5 points)
Define functions to process the responses, convert the letter (A-D) to an index (0-3), and compute accuracies.

In [ ]:
import re

def process_answer(answer):
    """
    Converts a letter-based multiple choice answer to a numerical index.

    Parameters:
        answer (str): The answer, expected to start with one of A, B, C, or D. We only care about the first letter.

    Returns:
        int: The index corresponding to the answer (A -> 0, B -> 1, C -> 2, D -> 3).
           Returns 4 if the answer is invalid.
    """
    # Check if the answer starts with A, B, C, or D (only accept capital letters). If so, return the corresponding index; else return 4
    """TO DO"""


def get_answers(responses, format):
    """
    Extracts answers from a list of LLM-generated responses and converts them to numeric indices.

    Parameters:
        responses (list of str): List of responses containing answers in the format.
        format (str): The format of the answer, 'note' or 'boxed'

    Returns:
        list of int: A list of numeric indices corresponding to the extracted answers.
                If no answer is found, 4 is used to indicate an invalid/missing answer.
    """
    # define searching pattern
    if format == "note":
        pattern = re.compile(r'The correct answer is:\s*(.*)')

    elif format == "boxed":
        pattern = re.compile(r'\\boxed\{([^}]+)\}')

    answers = []

    # Process each response
    for i, response in enumerate(responses):
        pattern_match = """TO DO"""
        if pattern_match:
            """TO DO"""
        else:
          """TO DO"""
          print(f"No answer found in format in response {i}")

    return answers

def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy by comparing true labels with predicted labels, and identifies indices of incorrect predictions.

    Parameters:
        y_true (list of int): Ground truth answer.
        y_pred (list of int): Predicted answer.

    Returns:
        accuracy (float): accuracy.
        wrong_indices (list of int): Indices where predictions are incorrect.
    """
    """TO DO"""
    return accuracy, wrong_indices

### 3.4 Process responses (2 points)
Process the LLM responses using the finction defined, print the accuracy and wrong indices for each model.

In [ ]:
# results for format 1: note
for model, responses in responses_1.items():
    print(f"Processing {model}...")
    llm_answers = """TO DO"""
    accuracy, wrong_id = """TO DO"""
    print(f"Accuracy: {accuracy}")
    print(f"Wrong IDs: {wrong_id}\n")


In [ ]:
# results for format 2: boxed
for model, responses in responses_2.items():
    print(f"Processing {model}...")
    llm_answers = """TO DO"""
    accuracy, wrong_id = """TO DO"""
    print(f"Accuracy: {accuracy}")
    print(f"Wrong IDs: {wrong_id}\n")

## Part 4: Questions (4 points)


### 1. Compare and comment on the performance of different models. (1 point)

your answer here

### 2. Explain the concept of the temperature parameter in LLM inference. How does adjusting the temperature affect generation behavior? In your opinion, what temperature setting is most appropriate when evaluating LLMs on multiple-choice questions, and why? (3 points)

your answer here